# Data Ingest

In [ ]:
import pandas as pd

FILE_LOCATION = '/kaggle/input/daily-website-visitors/daily-website-visitors.csv'

whole_dataset = pd.read_csv(FILE_LOCATION, 
                            index_col='Date',
                            thousands=',')
whole_dataset.index = pd.to_datetime(whole_dataset.index)
whole_dataset

In [ ]:
whole_dataset.info()

In [ ]:
whole_dataset.describe()

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(3, figsize=(12, 5))

axs[0].plot(whole_dataset['First.Time.Visits'])
axs[1].plot(whole_dataset['Unique.Visits'])
axs[2].plot(whole_dataset['Returning.Visits'])
plt.show()

# Preprocessing the data

* Target Attribute: **Returning.Visits** 
We shall predict the **Returning.Visits** given past data.


In [ ]:
target_column = whole_dataset['Returning.Visits']
target_column

In [ ]:
target_column.plot(figsize=(15, 3))
plt.show()

# Compute Train and Test Data Boundaries

In [ ]:
len(target_column)

In [ ]:
TEST_DATA_PERCENTAGE = 0.1

TEST_DATA_BOUNDARY_INDEX = int((1 - TEST_DATA_PERCENTAGE) * len(target_column))
print(f"Train data:\tReturning Visits [:{TEST_DATA_BOUNDARY_INDEX}] ({TEST_DATA_BOUNDARY_INDEX + 1})")
print(f"Test data:\tReturning Visits [{TEST_DATA_BOUNDARY_INDEX}:] ({len(target_column) - TEST_DATA_BOUNDARY_INDEX})")
print(f"\nLast target on train data: {target_column[TEST_DATA_BOUNDARY_INDEX]}")

In [ ]:
print(f"Train dataset ending values: {target_column[TEST_DATA_BOUNDARY_INDEX - 10: TEST_DATA_BOUNDARY_INDEX].values}")
print(f"Test dataset starting values: {target_column[TEST_DATA_BOUNDARY_INDEX: TEST_DATA_BOUNDARY_INDEX + 10].values}")

## Window-ize the dataset

In [ ]:
from tensorflow.keras.utils import timeseries_dataset_from_array

WINDOW_SIZE = 3
train_dataset = timeseries_dataset_from_array(target_column[:-WINDOW_SIZE], 
                                                 target_column[WINDOW_SIZE:], 
                                                 sequence_length=WINDOW_SIZE,
                                                 end_index=TEST_DATA_BOUNDARY_INDEX - 1)
len(train_dataset), len(list(train_dataset.unbatch()))

In [ ]:
target_column[TEST_DATA_BOUNDARY_INDEX-10:TEST_DATA_BOUNDARY_INDEX+10].values, (list(train_dataset)[-1][0][-1].numpy(), list(train_dataset)[-1][1][-1].numpy())

In [ ]:
test_dataset = timeseries_dataset_from_array(target_column[TEST_DATA_BOUNDARY_INDEX - WINDOW_SIZE:], 
                                                 target_column[TEST_DATA_BOUNDARY_INDEX:], 
                                                 sequence_length=WINDOW_SIZE
                                            )
len(test_dataset), len(list(test_dataset.unbatch()))

In [ ]:
target_column[TEST_DATA_BOUNDARY_INDEX-10:TEST_DATA_BOUNDARY_INDEX+10].values, list(test_dataset)[0][0][0].numpy(), list(test_dataset)[0][1][0].numpy()

In [ ]:
# First point in test dataset
list(test_dataset)[0][0][0].numpy(), list(test_dataset)[0][1][0].numpy()

In [ ]:
# Last point in test dataset
list(test_dataset)[-1][0][-1].numpy(), list(test_dataset)[-1][1][-1].numpy()

## Plot the train and test datasets

In [ ]:
import numpy as np 
import matplotlib.dates as mdates

def plot_time_series(predictions = None, start_index=1500):
    timesteps = pd.to_datetime(target_column.index)
    
    fig,ax = plt.subplots(1,figsize=(15,5))
    ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 7)))
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%b'))
    
    # Plot train dataset
    plt.plot(timesteps[start_index:TEST_DATA_BOUNDARY_INDEX], target_column[start_index:TEST_DATA_BOUNDARY_INDEX],
            color='blue')
    # Plot test dataset
    plt.plot(timesteps[TEST_DATA_BOUNDARY_INDEX:], target_column[TEST_DATA_BOUNDARY_INDEX:],
             color='green', linewidth=0.4)
    
    if predictions is not None:
        pred_timesteps = timesteps[TEST_DATA_BOUNDARY_INDEX:]
        plt.plot(pred_timesteps, predictions, linewidth=0.4, color='red')
        plt.scatter(pred_timesteps, predictions, s=0.4, color='red')

        
plot_time_series()

# Model 0: Baseline model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras import Model

class NaiveForecastLayer(Model):
    def __init__(self):
        super().__init__()
        
    def call(self, inputs):
        result = inputs[:, -1]
        return result[:, tf.newaxis]


In [ ]:
baseline_model = NaiveForecastLayer()
baseline_model._name = 'model_0'

baseline_model.compile(metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [ ]:
baseline_predictions = baseline_model.predict(test_dataset)

In [ ]:
plot_time_series(baseline_predictions.ravel(), start_index=1900)

In [ ]:
y_true = target_column[TEST_DATA_BOUNDARY_INDEX : ]

len(y_true), y_true

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

def evaluate_predictions(y_true, y_preds):
    mae = mean_absolute_error(y_true, y_preds)
    mse = mean_squared_error(y_true, y_preds)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(y_true, y_preds)
    
    return {
        'mae': mae,
        'mse': mse, 
        "rmse": rmse,
        "mape": mape
    }

evaluate_predictions(y_true, baseline_predictions)

In [ ]:
MODEL_METRICS = pd.DataFrame(columns=['mae', 'mse', 'rmse', 'mape'])

def evaluate_model(model):
    predictions = model.predict(test_dataset, verbose=0)
    metrics = evaluate_predictions(y_true, predictions)
    
    MODEL_METRICS.loc[model.name] = metrics
    plot_time_series(predictions.ravel(), start_index=1900)
    return metrics


In [ ]:
evaluate_model(baseline_model)

In [ ]:
MODEL_METRICS

# Model 1: Recurrent Network Model (GRU)

In [ ]:
from tensorflow.keras.layers import GRU, Dense, Input, Lambda
from tensorflow.keras import Sequential

tf.random.set_seed(42)
model_1 = Sequential([
    Input(shape=(WINDOW_SIZE,)),
    Lambda(lambda x: tf.expand_dims(x, axis=1)),
    GRU(128, activation="relu"),
    Dense(1)
], name='model_1')

model_1.compile(
    loss=tf.keras.losses.MeanAbsoluteError(),
    optimizer=tf.keras.optimizers.Adam()
)

model_1.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import os

def create_checkpoint_callback(model):
    filepath = os.path.join('models', model.name)
    return ModelCheckpoint(filepath, monitor='loss', save_weights_only=True, save_best_only=True)

model_1.fit(train_dataset, epochs=5, callbacks=[ create_checkpoint_callback(model_1) ])

In [ ]:
evaluate_model(model_1)

In [ ]:
MODEL_METRICS

# Model 3: Multi-input Model

In [ ]:
unbatched_train_dataset = whole_dataset[:TEST_DATA_BOUNDARY_INDEX + 1].copy()
unbatched_train_dataset

DA: Any significant different per day, per month, per year?

## Per `Day of Week` grouping

In [ ]:
dataset_by_day = unbatched_train_dataset.groupby(by=['Day'])
dataset_by_day['Returning.Visits'].mean()

In [ ]:
DAYS_OF_WEEK = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
pd.DataFrame(dataset_by_day['Returning.Visits'].mean()).loc[DAYS_OF_WEEK].plot(kind='bar')

In [ ]:
dataset_by_day['Returning.Visits'].hist(legend=True, alpha=0.5)
plt.show()


In [ ]:
import calendar

train_dataset_with_months = unbatched_train_dataset.copy()
train_dataset_with_months['Month.Name'] = pd.Series(train_dataset_with_months.index, 
                                                    index=train_dataset_with_months.index)\
                                            .apply(lambda x: calendar.month_name[x.month])
train_dataset_with_months

In [ ]:
MONTH_NAMES = list(calendar.month_name)[1:]
dataset_group_by_month = train_dataset_with_months.groupby(by='Month.Name')
dataset_group_by_month['Returning.Visits'].mean().loc[MONTH_NAMES]

In [ ]:
pd.DataFrame(dataset_group_by_month['Returning.Visits'].mean()).loc[MONTH_NAMES].plot(kind='bar')
plt.show()

## Prepare the dataset

In [ ]:
train_dataset_with_months

In [ ]:
dataset2 = train_dataset_with_months.copy()[['Day', 'Month.Name', 'Returning.Visits']]
dataset2

In [ ]:
def windowize_dataset(dataset):
    for i in range(WINDOW_SIZE):
        dataset[f'Returning.Visits[t-{i+1}]'] = dataset['Returning.Visits'].shift(periods=i+1)
    return dataset

dataset2 = windowize_dataset(dataset2.copy())
dataset2

In [ ]:
dataset2 = dataset2.dropna()
dataset2

In [ ]:
rv_cols = [f"Returning.Visits[t-{i+1}]" for i in range(WINDOW_SIZE)]

dataset2_rv_history_features = dataset2[rv_cols]
dataset2_rv_history_features

In [ ]:
dataset2_cat_features =  dataset2[['Day', 'Month.Name']]
dataset2_cat_features

In [ ]:
train_dataset2 = dataset2['Returning.Visits']
train_dataset2

## Building the model

In [ ]:
from tensorflow.keras.layers import Concatenate, Dropout

tf.random.set_seed(42)
def build_model_3():
    seq_input = Input(shape=(WINDOW_SIZE,))
    lambda_layer = Lambda(lambda x: x[:, tf.newaxis])(seq_input)
    rnn_layer = GRU(64, activation='relu')(lambda_layer)

    cat_input = Input(shape=(2,))
    cat_dense_layer = Dense(32, activation='relu')(cat_input)
    
    concat_layer = Concatenate()([rnn_layer, cat_dense_layer])
    dense_layer1 = Dense(128, activation='relu')(concat_layer)
    dropout_layer = Dropout(0.5)(dense_layer1)
    output_layer = Dense(1, activation='linear')(dropout_layer)
    
    return Model(inputs=[seq_input, cat_input], outputs=output_layer, name="model_3")
    
model_3 = build_model_3()
model_3.compile(
    loss=tf.keras.losses.MeanAbsoluteError(),
    optimizer=tf.keras.optimizers.Adam()
)

model_3.summary()

In [ ]:
tf.keras.utils.plot_model(model_3)

In [ ]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
X_cat_encoder = OrdinalEncoder(categories = [DAYS_OF_WEEK, MONTH_NAMES])
X_cat_encoded = X_cat_encoder.fit_transform(dataset2_cat_features)
X_cat_encoded, X_cat_encoder.categories_

In [ ]:
from tensorflow.data import Dataset

model3_history = model_3.fit(x=[dataset2_rv_history_features, X_cat_encoded], y=train_dataset2, epochs=5)
pd.DataFrame(model3_history.history).plot()

In [ ]:
test_dataset2 = windowize_dataset(whole_dataset[TEST_DATA_BOUNDARY_INDEX-WINDOW_SIZE:].copy())
test_dataset2['Month.Name'] = pd.Series(test_dataset2.index, index=test_dataset2.index)\
                        .apply(lambda x: calendar.month_name[x.month])
test_dataset2 = test_dataset2.dropna()
test_dataset2

In [ ]:
X_test_rv_history_input = test_dataset2[rv_cols]
X_test_rv_history_input

In [ ]:
X_test_cat_input = test_dataset2[['Day', 'Month.Name']]
X_test_cat_input = X_cat_encoder.transform(X_test_cat_input) 
X_test_cat_input.shape, X_test_cat_input[:5]

In [ ]:
model_3_preds = model_3.predict([X_test_rv_history_input, X_test_cat_input])
model_3_preds[:15]

In [ ]:
y_dataset = test_dataset2['Returning.Visits']
y_dataset

In [ ]:
def evaluate_model_predictions(y_true, predictions, model_name):
    metrics = evaluate_predictions(y_true, predictions)
    
    MODEL_METRICS.loc[model_name] = metrics
    plot_time_series(predictions.ravel(), start_index=1900)
    return metrics

evaluate_model_predictions(y_dataset, model_3_preds, 'model_3 (multi-input)')

In [ ]:
MODEL_METRICS

# Model 4: Ensemble methods

In [ ]:
def build_model_5(n_models, loss_fns):
    models = []
    for loss_fn in loss_fns:
        print(f"Training {n_models} models for {loss_fn} loss...")
        for i in range(n_models):
            model = Sequential([
                Input(shape=(WINDOW_SIZE,)),
                Lambda(lambda x: tf.expand_dims(x, axis=1)),
                GRU(128, activation='relu'),
                Dense(1, activation='linear')
            ])
            
            model.compile(loss=loss_fn, optimizer=tf.keras.optimizers.Adam())
            models.append(model)
            
    
    return models


model_5 = build_model_5(n_models=5, loss_fns=['mae', 'mse', 'mape'])
model_5

In [ ]:
model_5[0].summary()

In [ ]:
for i, model in enumerate(model_5):
    print(f"Training model {i+1} out of {len(model_5)} models")
    model.fit(train_dataset, epochs=5, verbose=0)

In [ ]:
def ensemble_prediction(models):
    predictions = []
    for model in models:
        pred = model.predict(test_dataset, verbose=0)
        predictions.append(pred)
    
    return np.array(predictions)

model_5_all_preds = ensemble_prediction(model_5)
model_5_all_preds.shape

In [ ]:
model_5_all_preds.shape

In [ ]:
def aggregate_ensemble_predictions(predictions):
    return tf.reduce_mean(predictions, axis=0).numpy()

model_5_preds = aggregate_ensemble_predictions(model_5_all_preds)
model_5_preds.shape

In [ ]:
evaluate_model_predictions(y_true, model_5_preds, 'model_5 (ensemble)')

In [ ]:
MODEL_METRICS